<a href="https://colab.research.google.com/github/coreymunk/Deep-Learning-CIFAR-100/blob/main/Simple_Neural_Net_CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import numpy as np
from keras.datasets import cifar100

# Load and preprocess CIFAR-100 data
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

In [3]:
# Normalize the images
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flatten the images
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# One-hot encoding of labels
def one_hot_encode(y, classes=100): 
    return np.eye(classes)[y.reshape(-1)]

y_train_one_hot = one_hot_encode(y_train)
y_test_one_hot = one_hot_encode(y_test)

# Initialize parameters
def init_params():
    number_of_input_features = 3072  # 32*32*3
    y_classes = 100
    num_hidden_neurons = 30  # This is adjustable

    W1 = np.random.rand(num_hidden_neurons, number_of_input_features) - 0.5
    b1 = np.random.rand(num_hidden_neurons, 1) - 0.5
    W2 = np.random.rand(y_classes, num_hidden_neurons) - 0.5
    b2 = np.random.rand(y_classes, 1) - 0.5

    return W1, b1, W2, b2

# Non-linearity functions
def ReLU(Z):
    return np.maximum(0, Z)

def ReLU_deriv(Z):
    return np.where(Z > 0, 1, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

# Forward propagation
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

# Backward propagation
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y, m):
    one_hot_Y = Y
    dZ2 = A2 - one_hot_Y
    dW2 = 1/m * dZ2.dot(A1.T)
    db2 = 1/m * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1, axis=1, keepdims=True)
    return dW1, db1, dW2, db2

# Update parameters
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

# Helper functions for predictions and accuracy
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

# Gradient descent function
def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    m = X.shape[1]  # Number of examples

    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y, m)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        if i % 100 == 0:
            predictions = get_predictions(A2)
            accuracy = get_accuracy(predictions, Y.argmax(axis=0))
            print(f"Iteration: {i}, Accuracy: {accuracy}")

    return W1, b1, W2, b2

# Training the model
# Consider using a subset of the data for quick testing
subset_size = 10000
X_sub = x_train_flat[:subset_size].T  # Transpose for correct shape
Y_sub = y_train_one_hot[:subset_size].T

W1, b1, W2, b2 = gradient_descent(X_sub, Y_sub, alpha=0.25, iterations=400)


Iteration: 0, Accuracy: 0.0103
Iteration: 100, Accuracy: 0.012
Iteration: 200, Accuracy: 0.0122
Iteration: 300, Accuracy: 0.0126
